<a href="https://colab.research.google.com/github/piratesdragon/homework_actual_problems/blob/main/homework_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1 (10 баллов)

Вам нужно воспроизвести пайплайн дообучения из семинара, используя русскоязычные модели. То есть, вам нужно дообучить генеративную модель генерировать более положительные тексты с помощью модели определения тональности.

1) Вам потребуется модель определения тональности для русского языка. Вы можете найти ее на huggingface или обучить самостоятельно. В прошлых семинарах мы пользовались датасетом с токсичными текстами - можно использовать его для обучения (или любой другой датасет на русском языке). Удобнее всего будет обучать модель через huggingface. Можете взять за основу вот этот туториал - https://huggingface.co/docs/transformers/training#train-a-tensorflow-model-with-keras После обучения можете сохранить модель через model.save_pretrained(path) и потом использовать ее как любую другую модель из huggingface hub.
Используйте не очень большую модель, чтобы все поместилось в колабе (например, distilbert-multilingual)

  2) Генеративную модель в целом можно взять любую, но лучше всего будет модель, специально обученная на русском языке (rugpt).

### Импорты

In [ ]:
%pip install datasets

In [ ]:
%pip install transformers trl wandb

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

### Подготовка датасета

In [6]:
dataset = load_dataset("csv", data_files='/content/drive/MyDrive/Вышка/labeled.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-85d004d83820a0bc/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['comment', 'toxic'],
        num_rows: 14412
    })
})

In [9]:
def build_dataset(config, dataset=dataset['train'], input_min_text_length=2, input_max_text_length=8):
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    ds = dataset
    # ds = ds.rename_columns({"comment": "review"})

    # еще есть фильтрация по длине
    ds = ds.filter(lambda x: len(x["comment"]) > 200, batched=False)
    ds = ds.filter(lambda x: len(x["comment"]) < 2000, batched=False)

    # длина кусочка определяется случайно
    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["comment"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [11]:
dataset = build_dataset(config)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Filter:   0%|          | 0/14412 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3530 [00:00<?, ? examples/s]

Map:   0%|          | 0/3473 [00:00<?, ? examples/s]

### Генерация текстов

In [ ]:
config = PPOConfig(
    model_name="ai-forever/rugpt3small_based_on_gpt2",
    learning_rate=1.41e-5,
    log_with=None,
    mini_batch_size=16
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

In [12]:
# active_model
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
# reference_model
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

In [14]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

### Модель для сентимент анализа

In [15]:
sentiment_pipe = pipeline("sentiment-analysis", model="MonoHime/rubert-base-cased-sentiment-new", device=device)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [16]:
text = "этот фильм ужасный!!"
sentiment_pipe(text, **sent_kwargs)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'NEUTRAL', 'score': -0.05972132086753845},
  {'label': 'POSITIVE', 'score': -1.0395042896270752},
  {'label': 'NEGATIVE', 'score': 1.3660616874694824}]]

In [17]:
text = "этот фильм такой классный!!"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'NEUTRAL', 'score': -1.3216748237609863},
  {'label': 'POSITIVE', 'score': 2.8044347763061523},
  {'label': 'NEGATIVE', 'score': -2.7516777515411377}]]

### Генеративная модель + модель для сентимент анализа

In [19]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader),
                         total=dataset.num_rows//ppo_trainer.dataloader.batch_sampler.batch_size):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

  0%|          | 0/13 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [21]:
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_colwidth', 5000)

In [26]:
#### get a batch from the dataset
bs = 16
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]




/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [27]:
final = pd.DataFrame.from_dict(game_data)
final

,query,response (before),response (after),rewards (before),rewards (after)
0,Предлага,ется изготовитель Бентли Уайт.,"ю своему кафе, попробуйте).\n\nСамая простая",-0.016216,3.234656
1,Встала м,ерою в городе Коломне,уха) Не подскажите в кам,-0.228733,-1.039451
2,для двигателей нужен,"один бочек, а в гараже для неодимового магни","грамотный специалист), будут замечательно работать.\n\nА вот",0.173746,3.235071
3,Все верно,". Российской молодежи от 18 до 23 лет, по бюллет",", всё верно).\n\n\n4739823",-0.198791,0.284039
4,У вас проблема с,функциональностью — можете попробовать IPTV. Во-,системой)?\nЗвучит заманчиво.\n\n,0.163991,3.233192
5,Кварплата кв. На,стройку LPC = ставка,стройке) \n 17.,-0.227443,0.223454
6,Ну ты блядь и тупой.,"Менталитету не обучен,",А я - про запас :)\n\n\n,-1.039357,3.235271
7,да я стараюсь,"всегда не выводить машины,",;)\n\nВ,-0.769100,-0.222715
8,Отчего же,"здесь, в забытьи, мне жутко?\nЛучше",в русской традиции такого серьёзного?)Pfwnfkh,-1.039650,3.235314
9,Ещё в студен,ческой группе самбу жался попзажом от трансформатор,"честве, когда учился в другом институте.), в итоге уснуть обыкновенно позволяет",0.189825,3.235116


Действительно, в большинстве случаев после генерации тексты более положительные. Из того, что прямо заметно - огромное количество скобочек и смайликов - что ожидаемо :)